In [9]:
import pandas as pd
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

# 1. Percorso
folder_path = r"C:\Users\andrea\Desktop\esercitazioni finali\Finale"

# 2. Lista dei file
file_names = [
    "@CL  TOP_VAR_20_TFM_CL_15_.xlsx",
    "@CL  TOP_VAR_33_TFM_CL_15_.xlsx",
    "@ES  TOP_VAR_27_TFM_ES_15_.xlsx",
    "@ES  TOP_VAR_29_TFM_ES_15_.xlsx",
    "@GC  TOP_VAR_17_TFM_GC_15_.xlsx",
    "@GC  TOP_VAR_24_TFM_GC_15_.xlsx",
    "@NQ  TOP_VAR_22_TFM_NQ_15_.xlsx",
    "@NQ  TOP_VAR_7_TFM_NQ_15_.xlsx",
]

# 3. Lista per contenere i dataframe
all_trades = []

# 4. Loop sui file
for file in file_names:
    file_path = os.path.join(folder_path, file)
    df = pd.read_excel(file_path)

    # Aggiungo info di strategia e asset
    df['Strategia'] = file.replace('.xlsx', '')
    if 'CL' in file:
        df['Asset'] = 'CL'
    elif 'ES' in file:
        df['Asset'] = 'ES'
    elif 'GC' in file:
        df['Asset'] = 'GC'
    elif 'NQ' in file:
        df['Asset'] = 'NQ'

    # Converto la colonna 'Date' in datetime
    df['Date'] = pd.to_datetime(df['Date'])

    # Aggiungo la colonna Periodo (Backtest o Forward)
    df['Periodo'] = df['Date'].apply(lambda x: 'Backtest' if x.year <= 2021 else 'Forward')

    # Converto da Mini a Micro dividendo per 10
    df['Profit ($)'] = df['Profit ($)'] / 10
    df['Drawdown ($)'] = df['Drawdown ($)'] / 10

    # Rinomino le colonne per uniformità (opzionale, ma utile)
    df.rename(columns={
        'Date': 'entry_date',
        'Profit ($)': 'profit',
        'Drawdown ($)': 'drawdown'
    }, inplace=True)

    # Aggiungo il dataframe alla lista
    all_trades.append(df)

# 5. Unisco tutti i dataframe
df_all = pd.concat(all_trades, ignore_index=True)

# 6. Salvo il risultato finale
df_all.to_csv("trades_micro_all.csv", index=False)

# 7. Mostro anteprima
df_all.head()


,Trade #,Order #,Type,Signal,entry_date,Time,Price,profit,drawdown,Strategia,Asset,Periodo
0,323.0,645,EntryShort,Short,2015-01-02,07:15:00,61.51,442.0,-57.0,@CL TOP_VAR_20_TFM_CL_15_,CL,Backtest
1,NaN,646,ExitShort,End of Day Exit,2015-01-07,17:00:00,57.09,NaN,NaN,@CL TOP_VAR_20_TFM_CL_15_,CL,Backtest
2,324.0,647,EntryShort,Short,2015-01-08,02:45:00,56.77,228.0,-112.0,@CL TOP_VAR_20_TFM_CL_15_,CL,Backtest
3,NaN,648,ExitShort,End of Day Exit,2015-01-13,17:00:00,54.49,NaN,NaN,@CL TOP_VAR_20_TFM_CL_15_,CL,Backtest
4,325.0,649,EntryLong,Buy,2015-01-27,02:30:00,52.74,41.0,-13.0,@CL TOP_VAR_20_TFM_CL_15_,CL,Backtest


In [2]:
# Raggruppo per Asset, Strategia, Periodo
grouped = df_all.groupby(['Asset', 'Strategia', 'Periodo'])

# Creo lista per salvare i risultati
results = []

# Loop su ogni gruppo (asset-strategia-periodo)
for (asset, strat, periodo), group in grouped:
    num_trades = len(group)
    total_profit = group['profit'].sum()
    mean_profit = group['profit'].mean()
    win_rate = (group['profit'] > 0).mean() * 100
    max_drawdown = group['drawdown'].min()  # drawdown è negativo
    profit_factor = group[group['profit'] > 0]['profit'].sum() / abs(group[group['profit'] < 0]['profit'].sum()) if (group['profit'] < 0).any() else float('inf')

    results.append({
        'Asset': asset,
        'Strategia': strat,
        'Periodo': periodo,
        'N. Trade': num_trades,
        'Profitto Totale (€)': round(total_profit, 2),
        'Profitto Medio (€)': round(mean_profit, 2),
        '% Trade Vincenti': round(win_rate, 2),
        'Max Drawdown (€)': round(max_drawdown, 2),
        'Profit Factor': round(profit_factor, 2)
    })

# Creo DataFrame con i risultati
df_metrics = pd.DataFrame(results)

# Mostro i primi risultati
df_metrics.sort_values(by=['Asset', 'Strategia', 'Periodo'], inplace=True)
df_metrics.head(20)


,Asset,Strategia,Periodo,N. Trade,Profitto Totale (€),Profitto Medio (€),% Trade Vincenti,Max Drawdown (€),Profit Factor
0,CL,@CL TOP_VAR_20_TFM_CL_15_,Backtest,913,14387.00,31.48,27.16,-299.0,1.52
1,CL,@CL TOP_VAR_20_TFM_CL_15_,Forward,443,5766.00,26.09,21.44,-205.0,1.28
2,CL,@CL TOP_VAR_33_TFM_CL_15_,Backtest,928,13199.00,28.45,25.97,-241.0,1.54
3,CL,@CL TOP_VAR_33_TFM_CL_15_,Forward,502,6223.00,24.79,23.31,-220.0,1.32
4,ES,@ES TOP_VAR_27_TFM_ES_15_,Backtest,1127,15556.25,27.58,25.55,-300.0,1.54
5,ES,@ES TOP_VAR_27_TFM_ES_15_,Forward,483,8457.50,35.09,24.02,-300.0,1.33
6,ES,@ES TOP_VAR_29_TFM_ES_15_,Backtest,981,19020.00,38.74,26.20,-361.2,1.68
7,ES,@ES TOP_VAR_29_TFM_ES_15_,Forward,447,10137.50,45.46,24.38,-280.0,1.42
8,GC,@GC TOP_VAR_17_TFM_GC_15_,Backtest,827,18128.00,43.79,26.72,-300.0,1.68
9,GC,@GC TOP_VAR_17_TFM_GC_15_,Forward,393,6182.00,31.54,23.66,-308.0,1.31


In [3]:
#salvo il csv
#df_metrics.to_csv("strategie_riepilogo.csv", index=False)

In [4]:
# Seleziono solo il periodo di Backtest
df_bkt = df_metrics[df_metrics['Periodo'] == 'Backtest'].copy()

# Normalizzazione su: Profitto, Drawdown, Profit Factor
scaler = MinMaxScaler()
df_bkt['profit_norm'] = scaler.fit_transform(df_bkt[['Profitto Totale (€)']])
df_bkt['dd_norm'] = scaler.fit_transform(df_bkt[['Max Drawdown (€)']].abs())
df_bkt['pf_norm'] = scaler.fit_transform(df_bkt[['Profit Factor']])

# Calcolo punteggio finale basato SOLO su backtest
df_bkt['Score'] = df_bkt['profit_norm'] + df_bkt['pf_norm'] - df_bkt['dd_norm']

# Ranking per asset
df_bkt['Ranking'] = df_bkt.groupby('Asset')['Score'].rank(ascending=False, method='min')

# Ordino e visualizzo
df_bkt.sort_values(by=['Asset', 'Ranking'], inplace=True)
df_bkt[['Asset', 'Strategia', 'Profitto Totale (€)', 'Profit Factor', 'Max Drawdown (€)', 'Score', 'Ranking']]


,Asset,Strategia,Profitto Totale (€),Profit Factor,Max Drawdown (€),Score,Ranking
2,CL,@CL TOP_VAR_33_TFM_CL_15_,13199.00,1.54,-241.0,0.714977,1.0
0,CL,@CL TOP_VAR_20_TFM_CL_15_,14387.00,1.52,-299.0,0.256772,2.0
6,ES,@ES TOP_VAR_29_TFM_ES_15_,19020.00,1.68,-361.2,0.444409,1.0
4,ES,@ES TOP_VAR_27_TFM_ES_15_,15556.25,1.54,-300.0,0.374148,2.0
8,GC,@GC TOP_VAR_17_TFM_GC_15_,18128.00,1.68,-300.0,0.896792,1.0
10,GC,@GC TOP_VAR_24_TFM_GC_15_,12037.00,1.57,-265.0,0.518281,2.0
14,NQ,@NQ TOP_VAR_7_TFM_NQ_15_,27750.00,1.29,-298.5,0.521631,1.0
12,NQ,@NQ TOP_VAR_22_TFM_NQ_15_,14214.50,1.37,-288.5,-0.051467,2.0


In [5]:
strategie_scelte = df_bkt[df_bkt['Ranking'] == 1]
strategie_scelte[['Asset', 'Strategia', 'Score']]


,Asset,Strategia,Score
2,CL,@CL TOP_VAR_33_TFM_CL_15_,0.714977
6,ES,@ES TOP_VAR_29_TFM_ES_15_,0.444409
8,GC,@GC TOP_VAR_17_TFM_GC_15_,0.896792
14,NQ,@NQ TOP_VAR_7_TFM_NQ_15_,0.521631


In [10]:
# 1. Seleziono solo periodo Forward
df_fwd = df_metrics[df_metrics['Periodo'] == 'Forward'].copy()

# 2. Recupero le strategie migliori nel Backtest
strategie_buone = df_bkt[df_bkt['Ranking'] == 1][['Asset', 'Strategia']].copy()
strategie_buone['Scelta'] = True

# 3. Unisco per segnare quali sono quelle scelte nel backtest
df_fwd_compare = df_fwd.merge(strategie_buone, on=['Asset', 'Strategia'], how='left')
df_fwd_compare['Scelta'] = df_fwd_compare['Scelta'].fillna(False)

# 4. Ordino per asset mettendo prima la strategia "scelta"
df_fwd_compare.sort_values(by=['Asset', 'Scelta'], ascending=[True, False], inplace=True)

# 5. Seleziono solo le colonne chiave
df_confronto = df_fwd_compare[['Asset', 'Strategia', 'Scelta', 'Profitto Totale (€)', 'Profitto Medio (€)', '% Trade Vincenti', 'Max Drawdown (€)', 'Profit Factor']]

# 6. Mostro il confronto
df_confronto


,Asset,Strategia,Scelta,Profitto Totale (€),Profitto Medio (€),% Trade Vincenti,Max Drawdown (€),Profit Factor
1,CL,@CL TOP_VAR_33_TFM_CL_15_,True,6223.0,24.79,23.31,-220.0,1.32
0,CL,@CL TOP_VAR_20_TFM_CL_15_,False,5766.0,26.09,21.44,-205.0,1.28
3,ES,@ES TOP_VAR_29_TFM_ES_15_,True,10137.5,45.46,24.38,-280.0,1.42
2,ES,@ES TOP_VAR_27_TFM_ES_15_,False,8457.5,35.09,24.02,-300.0,1.33
4,GC,@GC TOP_VAR_17_TFM_GC_15_,True,6182.0,31.54,23.66,-308.0,1.31
5,GC,@GC TOP_VAR_24_TFM_GC_15_,False,9874.0,63.70,25.08,-265.0,1.66
7,NQ,@NQ TOP_VAR_7_TFM_NQ_15_,True,13429.0,14.30,25.65,-1184.0,1.12
6,NQ,@NQ TOP_VAR_22_TFM_NQ_15_,False,9110.0,33.87,15.06,-245.0,1.22


In [11]:
#esport i csv
df_all.to_csv("trades_micro_all.csv", index=False)

df_metrics.to_csv("strategie_riepilogo.csv", index=False)

df_confronto.to_csv("confronto_forward_scelta_vs_altro.csv", index=False)

